### check and save arxiv_all_files

In [ ]:

import os
os.chdir('/mnt/data/ai4phys/sunyu/nougat')
save_fold = 'arxiv_all_files3'
arxiv_all_files = os.listdir(f'data/{save_fold}')
with open(f'data/pdf_list/{save_fold}.txt','w') as fi:
    fi.writelines([fold+'\n' for fold in arxiv_all_files])
len(arxiv_all_files)

In [13]:
# read arxiv_all_files
import os
os.chdir('/mnt/data/ai4phys/sunyu/nougat')
save_fold = 'arxiv_all_files3'
with open(f'data/pdf_list/{save_fold}.txt','r')as fi:
    arxiv_all_files = fi.read().splitlines()
print(len(arxiv_all_files))

11245


In [2]:
with open('data/pdf_list/latex_dir.txt','r') as fi:
    tex_folds = fi.readlines()
tex_folds = [t.strip() for t in tex_folds]
len(tex_folds)

1263122

In [7]:
# latex
count=0
for i in range(760000,810000):
    if tex_folds[i] in arxiv_all_files:
        count += 1
print(count,count/(810000-760000))

3755 0.0751


In [20]:
# latex2
count=0
for i in range(810000,850000):
    if tex_folds[i] in arxiv_all_files:
        count += 1
print(count,count/(850000-810000))

4674 0.11685


: 

In [9]:
# latex3
count=0
for i in range(850000,930000):
    if tex_folds[i] in arxiv_all_files:
        count += 1
print(count,count/(930000-850000))

6011 0.0751375


In [10]:
# latex4
count=0
for i in range(930000,970000):
    if tex_folds[i] in arxiv_all_files:
        count += 1
print(count,count/(970000-930000))

2834 0.07085


### 读取tex_fold/train.jsonl并合并

In [11]:
import subprocess
import json
def combine_data(tex_fold,save_fold_idx='3'):
    if os.path.exists(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/train.jsonl') and os.path.exists(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/png'):
        with open(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/train.jsonl','r') as fi:
            lines = fi.read()
        for png_file in os.listdir(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/png'):
            subprocess.run(['cp','-n',f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/png/{png_file}',f'data/arxiv_train_data/png/{tex_fold}_{png_file}'])
            # png改为相对路径
            lines = lines.replace(f'data/arxiv_color/{tex_fold}/png/{png_file}',f'png/{tex_fold}_{png_file}')
            # mask处理：编码为-1，在encoding时不加position_embedding
            lines = lines.replace('["mask"]','[[-1,-1],[-1,-1]]')
        with open(f'data/arxiv_train_data/train_color{save_fold_idx}.jsonl','a') as fo:
            fo.write(lines)
    elif os.path.exists(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/train_black.jsonl') and os.path.exists(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/png'):
        with open(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/train_black.jsonl','r') as fi:
            lines = fi.read() 
        for png_file in os.listdir(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/png'):
            subprocess.run(['cp','-n',f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/png/{png_file}',f'data/arxiv_train_data/png/{tex_fold}_{png_file}'])
            # png改为相对路径
            lines = lines.replace(f'data/arxiv_origin/{tex_fold}/png/{png_file}',f'png/{tex_fold}_{png_file}')
            # mask处理：编码为-1，在encoding时不加position_embedding
            lines = lines.replace('["mask"]','[[-1,-1],[-1,-1]]')
        with open(f'data/arxiv_train_data/train_black{save_fold_idx}.jsonl','a') as fo:
            fo.write(lines)

In [12]:
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor

with ThreadPoolExecutor(max_workers=8) as pool:  
    inputs = arxiv_all_files
    futures = pool.map(combine_data, inputs)
    

In [9]:
# process color lines
import json
import re
save_fold_idx='2'
with open(f'data/arxiv_train_data/train_color{save_fold_idx}.jsonl','r') as fi:
    lines = fi.readlines()
for i,line in enumerate(lines):
    dct = json.loads(line)
    for idx in range(len(dct['prompt'])):
        if dct['prompt'][idx] != [[-1,-1],[-1,-1]]:   # 找到第一个非'mask'位置
            break
    # 去掉开头多余符号
    if idx>0 and idx<len(dct['prompt'])-1:
        if '#' in dct['pretext'][idx-1]:   # 保留标题
            dct['prompt'] = dct['prompt'][idx-1:]
            dct['pretext'] = dct['pretext'][idx-1:]
        elif idx>=2 and ('#' in dct['pretext'][idx-2] or re.search(r'figure|table',dct['pretext'][idx-2],re.I)):   # 保留图表头
            dct['prompt'] = dct['prompt'][idx-2:]
            dct['pretext'] = dct['pretext'][idx-2:]
        else:
            dct['prompt'] = dct['prompt'][idx:]
            dct['pretext'] = dct['pretext'][idx:]
    lines[i] = json.dumps(dct)+'\n'
with open(f'data/arxiv_train_data/train_color{save_fold_idx}.jsonl','w') as fo:
    fo.writelines(lines)

In [10]:
# process black lines
import json
save_fold_idx='2'
with open(f'data/arxiv_train_data/train_black{save_fold_idx}.jsonl','r') as fi:
    lines = fi.readlines()
for i,line in enumerate(lines):
    dct = json.loads(line)
    dct['pretext'] = [dct['pretext']]
    dct['prompt'] = [dct['prompt']]
    lines[i] = json.dumps(dct)+'\n'
with open(f'data/arxiv_train_data/train_black{save_fold_idx}.jsonl','w') as fo:
    fo.writelines(lines)

In [12]:
# construct train.jsonl val.jsonl
import random
save_fold_idx='2'
with open(f'data/arxiv_train_data/train_color{save_fold_idx}.jsonl','r') as fi:
    color_lines = fi.readlines()
with open(f'data/arxiv_train_data/train_black{save_fold_idx}.jsonl','r') as fi:
    black_lines = fi.readlines()
total = 2*len(color_lines)
train,val = int(0.8*total),int(0.2*total)
train_lines = color_lines[:int(train/2)]+black_lines[:int(train/2)]
val_lines = color_lines[int(train/2):]+black_lines[int(train/2):int(total/2)]
random.shuffle(train_lines)
random.shuffle(val_lines)
with open(f'data/arxiv_train_data/train{save_fold_idx}.jsonl','w') as fo:
    fo.writelines(train_lines)
with open(f'data/arxiv_train_data/validation{save_fold_idx}.jsonl','w') as fo:
    fo.writelines(val_lines)

In [15]:
import os
import json
save_fold_idx='3'
os.chdir('/mnt/data/ai4phys/sunyu/nougat')
with open(f'data/arxiv_train_data/train_color{save_fold_idx}.jsonl','r') as fi:
    print(len(fi.readlines()))
with open(f'data/arxiv_train_data/train_black{save_fold_idx}.jsonl','r') as fi:
    print(len(fi.readlines()))

9403
8978


In [6]:
train = 182328+39586+15460
(2326-4280)/(24290-61716)

0.05220969379575696

: 

### remove useless folders

In [5]:
# remove bugs
import subprocess
import json
# train0.jsonl:0-34w = 9/34 = 0.26
# train1.jsonl:34-55w = 4/21 = 0.1
# train2.jsonl:55-76w = 0.8/21 = 0.03
# train3.jsonl:76-97w
save_fold_idx='3'
color_folds=0
black_folds=0
for tex_fold in tex_folds[760000:970000]:
    if os.path.exists(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}'):
        if os.path.exists(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/train.jsonl'):
            color_folds += 1
            continue
        elif os.path.exists(f'data/arxiv_all_files{save_fold_idx}/{tex_fold}/train_black.jsonl'):
            black_folds += 1
            continue
        # else:
        #     os.system(f'rm -r data/arxiv_all_files{save_fold_idx}/{tex_fold}')
print(color_folds,black_folds)

4280 8027
